In [1]:
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from scipy.linalg import toeplitz
from scipy.stats import norm
from copy import deepcopy
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py

In [2]:
init_notebook_mode(connected=True)

In [3]:
np.random.seed(12)

In [4]:
M = 30  # window lenght of SSA
N = 200  #length of the time series
T = 22  # period of the sine function
stdnoise = 0.1  # noise-to-signal ratio

In [5]:
t = np.arange(1, N+1)
X1 = np.sin(2*np.pi*t/T)
X2 = np.cos(2*np.pi*t/T)**3
noise = norm.ppf(np.random.rand(N,1))
noise = np.hstack((noise, norm.ppf(np.random.rand(N,1))))
noise = stdnoise*noise

In [6]:
X1 = X1 + noise[:, 0]
X2 = X2 + noise[:, 1]
X1 = X1 - np.mean(X1) 
X2 = X2 - np.mean(X2)
X1 = X1/np.std(X1)
X2 = X2/np.std(X2)
X = np.transpose([X1, X2])

In [7]:
trace1 = go.Scatter(
    x = t,
    y = X1
)
trace2 = go.Scatter(
    x = t,
    y = X2
)

fig = tools.make_subplots(rows=1, cols=2)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout'].update(height=600, width=900, title='timeseries')
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



### Calculate the covariance matrix

In [8]:
Y1 = np.zeros((N-M+1,M))
Y2 = np.zeros((N-M+1,M))
for m in range(M):
    Y1[:, m] = X1[m:N-M+1+m]
    Y2[:, m] = X2[m:N-M+1+m]
Y = Y1
Y = np.hstack((Y1, Y2))
Cemb = np.dot(np.transpose(Y),Y) / (N-M+1)

In [9]:
trace = go.Heatmap(z=Cemb)
data=[trace]
iplot(data)

### Calculate eigenvalues and eigenvectors

In [10]:
eig_val,eig_vec = np.linalg.eigh(Cemb)
indexes = np.argsort(eig_val)[::-1]
eig_val = np.array([eig_val[i] for i in indexes])
eig_vec = np.transpose([eig_vec[:, i] for i in indexes])

In [11]:
trace1 = go.Scatter(
    x = np.arange(len(eig_val)),
    y = eig_val
)
trace2 = go.Scatter(
    x = np.arange(len(eig_val)),
    y = eig_vec[:,0]
)
trace3 = go.Scatter(
    x = np.arange(len(eig_val)),
    y = eig_vec[:,1]
)
trace4 = go.Scatter(
    x = np.arange(len(eig_val)),
    y = eig_vec[:,2]
)
trace5 = go.Scatter(
    x = np.arange(len(eig_val)),
    y = eig_vec[:,3]
)

fig = tools.make_subplots(rows=3, cols=1)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 3, 1)
fig.append_trace(trace5, 3, 1)

fig['layout'].update(height=600, width=900, title='eigenvectors and eignevalues')
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]



### Calculate principal components

In [12]:
pc = np.dot(Y, eig_vec)

In [13]:
fig = tools.make_subplots(rows=4, cols=1)
for k in range(4):
    data = pc[:, k]
    trace = go.Scatter(
        x = np.arange(len(data)),
        y = data
    )
    fig.append_trace(trace, k+1, 1)
fig['layout'].update(height=600, width=900, title='principal components')
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



### Calculate reconstrcuted components

In [14]:
RC1 = np.zeros((N, 2*M))
RC2 = np.zeros((N, 2*M))
for m in range(2*M):
    buf1 = np.dot(np.transpose([pc[:, m]]), [np.transpose(eig_vec[:M, m])])
    buf1 = np.flipud(buf1)
    
    buf2 = np.dot(np.transpose([pc[:, m]]), [np.transpose(eig_vec[M:, m])])
    buf2 = np.flipud(buf2)
    
    for n in range(N):
        RC1[n, m] = np.mean(np.diag(buf1, -(N-M) + n))
        RC2[n, m] = np.mean(np.diag(buf2, -(N-M) + n))

In [15]:
fig = tools.make_subplots(rows=4, cols=2)
for k in range(4):
    data = RC1[:, k]
    trace1 = go.Scatter(
        x = np.arange(len(data)),
        y = data
    )
    
    data = RC2[:, k]
    trace2 = go.Scatter(
        x = np.arange(len(data)),
        y = data
    )
    
    fig.append_trace(trace1, k+1, 1)
    fig.append_trace(trace2, k+1, 2)
fig['layout'].update(height=600, width=900, title='reconstructed components')
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]
[ (4,1) x7,y7 ]  [ (4,2) x8,y8 ]



### Compare resconstructed and original time series

In [16]:
fig = tools.make_subplots(rows=2, cols=2)
trace1 = go.Scatter(
        x = t,
        y = X1
    )
trace2 = go.Scatter(
        x = t,
        y = np.sum(RC1,1)
    )
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)

trace3 = go.Scatter(
        x = t,
        y = X2
    )
trace4 = go.Scatter(
        x = t,
        y = np.sum(RC2,1)
    )
fig.append_trace(trace3, 1, 2)
fig.append_trace(trace4, 1, 2)

trace5 = go.Scatter(
        x = t,
        y = np.sum(RC1[:,:2],1)
    )
fig.append_trace(trace1, 2, 1)
fig.append_trace(trace5, 2, 1)

trace6 = go.Scatter(
        x = t,
        y = np.sum(RC2[:,:2],1)
    )
fig.append_trace(trace3, 2, 2)
fig.append_trace(trace6, 2, 2)

fig['layout'].update(height=600, width=900, title='original versus reconstruction')
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]

